In [24]:
# Step 1: Import necessary libraries
import os
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, Input
from tensorflow.keras.models import Sequential
import numpy as np
import librosa

In [26]:
# Step 2: Create a directory called 'simple_model' if it doesn't exist
os.makedirs("simple_model", exist_ok=True)

In [27]:
# Step 3: Define the feature extraction function
def extract_features(file_path, n_mfcc=13, max_pad_len=100):
    """Extract MFCC features from audio file"""
    try:
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=n_mfcc)
        pad_width = max_pad_len - mfccs.shape[1]
        if pad_width > 0:
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mfccs = mfccs[:, :max_pad_len]
        return mfccs.T  # Transpose to get (timesteps, features)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

In [28]:
# Step 4: Load and preprocess the audio dataset
dataset_dir = '../dataset'
audio_files = [os.path.join(dataset_dir, f) for f in os.listdir(dataset_dir) if f.endswith('.wav')]

# Extract features and filter out any that failed to process
features = [extract_features(file) for file in audio_files]
features = [f for f in features if f is not None and f.shape == (100, 13)]  # Ensuring consistent shapes
features = np.array(features)


In [29]:
# Verify the final shape of the features array
print("Features shape:", features.shape)

Features shape: (615, 100, 13)


In [30]:
# Step 5: Create labels (assuming binary classification for simplicity)
# Adjust this based on your own criteria, for example, based on file names
labels = np.array([0 if 'slow' in f else 1 for f in audio_files[:len(features)]])


In [31]:
# Step 6: Save the features and labels in 'simple_model' for future use
np.save("simple_model/features.npy", features)
np.save("simple_model/labels.npy", labels)

In [32]:
# Step 7: Build a simple RNN model
input_shape = (features.shape[1], features.shape[2])  # (timesteps, features)

# Create the model with custom names
model = Sequential([
    Input(shape=input_shape, name='input_layer'),  # Custom input name
    SimpleRNN(128, return_sequences=False, name='rnn_layer'),
    Dropout(0.5, name='dropout_layer'),
    Dense(64, activation='relu', name='dense_layer'),
    Dense(1, activation='sigmoid', name='output_layer')  # Custom output name
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Save the model
model_path = "simple_model/simple_rnn_model.keras"
model.save(model_path)
print(f"Model saved to {model_path}")


Model saved to simple_model/simple_rnn_model.keras


In [33]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [34]:
# Step 8: Train the model
model.fit(features, labels, epochs=10, batch_size=16, validation_split=0.2)

Epoch 1/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7047 - loss: 0.5633 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 1.0000 - val_loss: 5.3316e-04
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0010 - val_accuracy: 1.0000 - val_loss: 4.1421e-04
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 9.2743e-04 - val_accuracy: 1.0000 - val_loss: 3.3744e-04
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 6.6137e-04 - val_accuracy: 1.0000 - val_loss: 2.7728e-04
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 5.7090e-04 - val_accuracy: 1.0000 - val_loss: 2.2675e-04
Epoch 7/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 5.2834e-04 - val_accuracy: 1.0000 - val_loss: 1.8363e-04
Epoch 8/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 

In [35]:
# Updated code
model_path = "simple_model/simple_rnn_model.keras"
model.save(model_path)

print(f"Model saved to {model_path}")


Model saved to simple_model/simple_rnn_model.keras


In [36]:
# Step 7: Build a simple RNN model
input_shape = (features.shape[1], features.shape[2])  # (timesteps, features)
model = Sequential([
    SimpleRNN(128, input_shape=input_shape, return_sequences=False),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

/Users/rrenoir/miniforge3/envs/tf-macos-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [38]:
# Step 8: Save the model with the new naming convention
model_path_keras = "simple_model/simple_rnn_model.keras"
model.save(model_path_keras)  # Save as Keras model

print(f"Model saved to {model_path_keras}")

# Step 9: Export as a TensorFlow SavedModel
tf_saved_model_path = "simple_model/tf_saved_model"
tf.saved_model.save(model, tf_saved_model_path)  # Save as TensorFlow SavedModel

print(f"Model exported as TensorFlow SavedModel at {tf_saved_model_path}")






Model saved to simple_model/simple_rnn_model.keras
INFO:tensorflow:Assets written to: simple_model/tf_saved_model/assets


INFO:tensorflow:Assets written to: simple_model/tf_saved_model/assets


Model exported as TensorFlow SavedModel at simple_model/tf_saved_model


In [22]:
pip install tensorflowjs


Note: you may need to restart the kernel to use updated packages.


In [39]:
import subprocess

subprocess.run([
    "tensorflowjs_converter",
    "--input_format=tf_saved_model",
    "--output_format=tfjs_graph_model",
    "simple_model/tf_saved_model",
    "simple_model/tfjs_model"
])


I0000 00:00:1729517582.873903 3643892 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)


CompletedProcess(args=['tensorflowjs_converter', '--input_format=tf_saved_model', '--output_format=tfjs_graph_model', 'simple_model/tf_saved_model', 'simple_model/tfjs_model'], returncode=0)